In [ ]:
import rootutils

rootutils.setup_root(
    search_from='.',
    indicator=[".git", "pyproject.toml"],
    project_root_env_var=True, # set the PROJECT_ROOT environment variable to root directory
    dotenv=True, # load environment variables from .env if exists in root directory
    pythonpath=True, # add root directory to the PYTHONPATH (helps with imports)
    cwd=True, # change current working directory to the root directory (helps with filepaths)
)

In [ ]:
import h5py

filepath = "data/datasets/emojis/animal_emojis_n_variants_0.h5"

with h5py.File(filepath, "r") as f:
    images = f["images"][:]
    descriptions = [d.item().decode("utf8") for d in f["descriptions"]]

print(images.shape)
example_text = descriptions[0]

In [ ]:
import torch
import open_clip
from transformers import AutoModel, AutoTokenizer

torch.set_grad_enabled(False)

MODEL = 'CLIP'

if MODEL == "OPENCLIP":
    model_name = "ViT-L-14"
    pretrained = "openai"
    model, _, preprocess = open_clip.create_model_and_transforms(model_name, pretrained=pretrained)
    tokenizer = open_clip.get_tokenizer('ViT-B-32')
    # emb_model_text = CLIPTextModelWithProjection.from_pretrained("openai/clip-vit-base-patch32")
    # tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")
elif MODEL == "CLIP":
    model = AutoModel.from_pretrained("openai/clip-vit-base-patch32")
    tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")
# elif MODEL == "BLIP":
#     model_name = "Salesforce/blip2-opt-2.7b"
#     processor = Auto.from_pretrained(model_name)
#     model = Blip2Model.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

In [ ]:
# # OPENCLIP
# print(example_text)
# text = tokenizer(example_text)
# print(text)
# bpe = tokenizer.decode(text[0].tolist())
# print(bpe)

In [ ]:
# CLIP
encoding = tokenizer(example_text)
print(encoding['input_ids'][1:8])
i = 1
tokenizer.decode(encoding['input_ids'][1:2])

In [ ]:
encoding = tokenizer(descriptions, padding=True, truncation=True, return_tensors="pt")
outputs = model.text_model(encoding['input_ids'], output_hidden_states=True)
outputs.last_hidden_state.shape